<a href="https://colab.research.google.com/github/ayulockin/debugNNwithWandB/blob/master/MNIST_pytorch_wandb_Vanishing_Gradient.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Imports and Setups

In [0]:
!pip install wandb -q

     |████████████████████████████████| 1.4MB 5.0MB/s 
     |████████████████████████████████| 102kB 14.4MB/s 
     |████████████████████████████████| 102kB 14.9MB/s 
     |████████████████████████████████| 92kB 13.6MB/s 
     |████████████████████████████████| 460kB 61.4MB/s 
     |████████████████████████████████| 71kB 11.9MB/s 
     |████████████████████████████████| 71kB 9.0MB/s 


In [0]:
import wandb

In [0]:
!wandb login

wandb: You can find your API key in your browser here: https://app.wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter: 69f60a7711ce6b8bbae91ac6d15e45d6b1f1430e
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
Successfully logged in to Weights & Biases!


In [0]:
import torch
from torch import nn
from torch import optim
from torch.nn import functional as F
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

import matplotlib.pyplot as plt
import numpy as np

#### For GPU

In [0]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


## MNIST Hand written Dataset

In [0]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.1307,), (0.3081,))])

trainset = torchvision.datasets.MNIST(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

testset = torchvision.datasets.MNIST(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

0it [00:00, ?it/s]

9920512it [00:01, 9253332.02it/s]                            


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw


  0%|          | 0/28881 [00:00<?, ?it/s]

32768it [00:00, 141591.26it/s]           
  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw


1654784it [00:00, 2271908.60it/s]                            
0it [00:00, ?it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw


8192it [00:00, 52847.07it/s]            

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw
Processing...
Done!


In [0]:
classes = ('0', '1', '2', '3', '4', '5', '6', '7', '8', '9')

## Models

1.   `NetwithIssue`: This model uses `sigmoid` as activation function in each layer. Earlier when `relu` was not discovered `sigmoid` created the problem of **Vanishing Gradient**. Due to this deep networks were not possible and it took longer training time to converge. 
2.   `Net`: This model uses `relu` as activation function in each layer. This activation solves the problem of vanishing gradient.



In [0]:
class NetwithIssue(nn.Module):
    def __init__(self):
        super(NetwithIssue, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1, bias=False)
        self.conv2 = nn.Conv2d(32, 64, 3, 1, bias=False)

        self.fc1 = nn.Linear(9216, 128, bias=False)
        self.fc2 = nn.Linear(128, 10, bias=False)

    def forward(self, x):
        ## Conv 1st Block
        x = self.conv1(x)
        x = torch.sigmoid(x) ## Notice
        x = self.conv2(x)
        x = torch.sigmoid(x) ## Notice
        x = F.max_pool2d(x, 2)

        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = torch.sigmoid(x) ## Notice
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1, bias=False)
        self.conv2 = nn.Conv2d(32, 64, 3, 1, bias=False)

        self.fc1 = nn.Linear(9216, 128, bias=False)
        self.fc2 = nn.Linear(128, 10, bias=False)

    def forward(self, x):
        ## Conv 1st Block
        x = self.conv1(x)
        x = F.relu(x) ## Notice
        x = self.conv2(x)
        x = F.relu(x) ## Notice
        x = F.max_pool2d(x, 2)

        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x) ## Notice
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

In [0]:
def train(model, device, train_loader, optimizer, epoch, steps_per_epoch=20):
  # Switch model to training mode. This is necessary for layers like dropout, batchnorm etc which behave differently in training and evaluation mode
  model.train()
  train_total = 0
  train_correct = 0

  # We loop over the data iterator, and feed the inputs to the network and adjust the weights.
  for batch_idx, (data, target) in enumerate(train_loader, start=0):
    if batch_idx > steps_per_epoch:
      break
    # Load the input features and labels from the training dataset
    data, target = data.to(device), target.to(device)
    
    # Reset the gradients to 0 for all learnable weight parameters
    optimizer.zero_grad()
    
    # Forward pass: Pass image data from training dataset, make predictions about class image belongs to (0-9 in this case)
    output = model(data)
    
    # Define our loss function, and compute the loss
    loss = F.nll_loss(output, target)

    scores, predictions = torch.max(output.data, 1)
    train_total += target.size(0)
    train_correct += int(sum(predictions == target))
            
    # Backward pass: compute the gradients of the loss w.r.t. the model's parameters
    loss.backward()
    
    # Update the neural network weights
    optimizer.step()

  acc = round((train_correct / train_total) * 100, 2)
  print('Epoch [{}], Loss: {}, Accuracy: {}, '.format(epoch, loss.item(), acc), end='')
  wandb.log({'Train Loss': loss.item(), 'Train Accuracy': acc})


In [0]:
def test(model, device, test_loader, classes):
  # Switch model to evaluation mode. This is necessary for layers like dropout, batchnorm etc which behave differently in training and evaluation mode
  model.eval()
  
  test_loss = 0
  test_total = 0
  test_correct = 0

  with torch.no_grad():
      for data, target in test_loader:
          # Load the input features and labels from the test dataset
          data, target = data.to(device), target.to(device)
          
          # Make predictions: Pass image data from test dataset, make predictions about class image belongs to (0-9 in this case)
          output = model(data)
          
          # Compute the loss sum up batch loss
          test_loss += F.nll_loss(output, target, reduction='sum').item()
          
          scores, predictions = torch.max(output.data, 1)
          test_total += target.size(0)
          test_correct += int(sum(predictions == target))
          
  acc = round((test_correct / test_total) * 100, 2)
  print(' Test_loss: {}, Test_accuracy: {}'.format(test_loss/test_total, acc))
  wandb.log({'Test Loss': test_loss/test_total, 'Test Accuracy': acc})


## Vanishing Gradient (Network with Issue)

In [0]:
net = NetwithIssue().to(device)
print(net)

optimizer = optim.Adam(net.parameters())

NetwithIssue(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), bias=False)
  (fc1): Linear(in_features=9216, out_features=128, bias=False)
  (fc2): Linear(in_features=128, out_features=10, bias=False)
)


In [0]:
wandb.init(project='debuggingnn')
wandb.watch(net, log='all')

for epoch in range(10):
  train(net, device, trainloader, optimizer, epoch)
  test(net, device, testloader, classes)

print('Finished Training')

Epoch [0], Loss: 2.3370118141174316, Accuracy: 9.38,  Test_loss: 2.3214688762664797, Test_accuracy: 10.1
Epoch [1], Loss: 2.302501916885376, Accuracy: 10.34,  Test_loss: 2.3092262718200685, Test_accuracy: 9.82
Epoch [2], Loss: 2.306159734725952, Accuracy: 8.33,  Test_loss: 2.303833694076538, Test_accuracy: 10.1
Epoch [3], Loss: 2.3103182315826416, Accuracy: 8.56,  Test_loss: 2.3044946189880373, Test_accuracy: 10.28
Epoch [4], Loss: 2.3062429428100586, Accuracy: 11.01,  Test_loss: 2.30686653175354, Test_accuracy: 10.09
Epoch [5], Loss: 2.318131923675537, Accuracy: 10.12,  Test_loss: 2.303500179672241, Test_accuracy: 11.35
Epoch [6], Loss: 2.302870273590088, Accuracy: 9.97,  Test_loss: 2.307649087905884, Test_accuracy: 8.92
Epoch [7], Loss: 2.3304078578948975, Accuracy: 10.12,  Test_loss: 2.30667571144104, Test_accuracy: 10.32
Epoch [8], Loss: 2.3118438720703125, Accuracy: 9.23,  Test_loss: 2.3033378204345705, Test_accuracy: 11.35
Epoch [9], Loss: 2.314885377883911, Accuracy: 9.6,  Test_

> Our model couldn't converge. Check out the gradients of each layer on the W&B run page.


In [0]:
del net

In [0]:
net = NetwithIssue().to(device)
optimizer = optim.Adam(net.parameters())

wandb.init(project='debuggingnn')
wandb.watch(net, log='all')

for epoch in range(40):
  train(net, device, trainloader, optimizer, epoch)
  test(net, device, testloader, classes)

print('Finished Training')

Epoch [0], Loss: 2.3126368522644043, Accuracy: 8.93,  Test_loss: 2.3131574661254883, Test_accuracy: 9.8
Epoch [1], Loss: 2.3287031650543213, Accuracy: 10.19,  Test_loss: 2.303910011672974, Test_accuracy: 10.28
Epoch [2], Loss: 2.2527520656585693, Accuracy: 16.74,  Test_loss: 2.2425626552581788, Test_accuracy: 18.93
Epoch [3], Loss: 2.14333438873291, Accuracy: 26.79,  Test_loss: 2.0964103660583495, Test_accuracy: 56.37
Epoch [4], Loss: 1.8815544843673706, Accuracy: 52.38,  Test_loss: 1.913255361557007, Test_accuracy: 43.34
Epoch [5], Loss: 1.662744402885437, Accuracy: 62.43,  Test_loss: 1.666070950126648, Test_accuracy: 68.16
Epoch [6], Loss: 1.5230214595794678, Accuracy: 72.69,  Test_loss: 1.478899013710022, Test_accuracy: 78.22
Epoch [7], Loss: 1.2913689613342285, Accuracy: 79.99,  Test_loss: 1.3357901279449462, Test_accuracy: 79.82
Epoch [8], Loss: 1.2442892789840698, Accuracy: 80.06,  Test_loss: 1.1794730913162232, Test_accuracy: 83.65
Epoch [9], Loss: 1.0750553607940674, Accuracy: 

## Relu saves the day

In [0]:
del net

net = Net().to(device)
print(net)

optimizer = optim.Adam(net.parameters())

Net(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), bias=False)
  (fc1): Linear(in_features=9216, out_features=128, bias=False)
  (fc2): Linear(in_features=128, out_features=10, bias=False)
)


In [0]:
wandb.init(project='debuggingnn')
wandb.watch(net, log='all')

for epoch in range(10):
  train(net, device, trainloader, optimizer, epoch)
  test(net, device, testloader, classes)

print('Finished Training')

Epoch [0], Loss: 0.4942484498023987, Accuracy: 69.27,  Test_loss: 0.4474091837644577, Test_accuracy: 86.37
Epoch [1], Loss: 0.29710474610328674, Accuracy: 88.47,  Test_loss: 0.2309967631816864, Test_accuracy: 93.38
Epoch [2], Loss: 0.14050628244876862, Accuracy: 92.49,  Test_loss: 0.19611059620380403, Test_accuracy: 93.99
Epoch [3], Loss: 0.11804260313510895, Accuracy: 94.94,  Test_loss: 0.1683878321647644, Test_accuracy: 94.79
Epoch [4], Loss: 0.08817225694656372, Accuracy: 95.61,  Test_loss: 0.15559489104747773, Test_accuracy: 95.51
Epoch [5], Loss: 0.035265251994132996, Accuracy: 95.68,  Test_loss: 0.1284967087507248, Test_accuracy: 96.07
Epoch [6], Loss: 0.22864656150341034, Accuracy: 95.54,  Test_loss: 0.11613762941360474, Test_accuracy: 96.56
Epoch [7], Loss: 0.136811763048172, Accuracy: 97.17,  Test_loss: 0.10356810202598572, Test_accuracy: 96.91
Epoch [8], Loss: 0.1875208616256714, Accuracy: 96.95,  Test_loss: 0.11141893649101257, Test_accuracy: 96.51
Epoch [9], Loss: 0.1007675

> The model converged quickly. Through the gradient plot it can be shown clearly that the gradients are distributed well. 